In [ ]:
import math
import numpy as np

import time
from numba import jit

from quad_funcs import *

import pickle

def quadratic_sieve(n, B=None, verbose=False, save=False):
    

    if B is None: B = math.ceil(math.exp(math.sqrt(math.log(n) * math.log(math.log(n))))) #tuned choice for B 
    
    if verbose: print("B =", B)
    

    

    
    print("finding primes...")
    
    b_primes = erat_sieve(B) #Finds all primes <= B using the sieve of Erasthenes
    
    K = len(b_primes)
    
    b_prod = reduce(mul, b_primes, 1) #Computes the product of all B-primes (to be used later)
    
    print("finding a_i's....")
    #A1 - find ± a_i where a_i^2 = n (mod p_i)
    a_s = [] 
    for p in tqdm(b_primes[1:]): #what to do for 2?
        a_s.append(find_a(n % p, p))
    
    
    
    
    ## Trying different x values
    
    print("sieving...")
    
    x = round(math.sqrt(n)) #Begins sieving at sqrt(n)
    x_initial = x
    
    smooths = []
    smooths_2 = []
    
    S = []
    
    widgets = ['b-smooth count: ', Percentage(), ' ', Bar(marker='-',left='[',right=']'),
           ' ', ETA(), ' ']

    pbar = ProgressBar(widgets=widgets, maxval=K+1) #a progress bar to keep track of how many b-smooth numbers have been found
    pbar.start()
    print(x)
    
    while len(smooths) <= K:
        #t = [(0, time.time())]
    
        y = (x**2) % n
        
        #t.append((1, time.time()))
        
        if is_bsmooth(b_prod, y): #checks if y = x^2 (mod n) is bsmooth
            #t.append((2, time.time()))
            
            factors = b_factor(b_primes, y) #factors y using trial division
            
            #t.append((3, time.time()))
            
            
            if verbose: print("x =", x, ": ", factors)
                
            S.append((x, x**2 - n))
            
            smooths.append(factors) #stores the exponents of factors of all bsmooth numbers in this array
            
            #t.append((4, time.time()))
            
        
        #t.append((5, time.time()))
        
        #for r in range(1, len(t)):
        #    print(t[r-1][0], "-", t[r][0], ": ", t[r][1] - t[r-1][1], end=", ")
        #print("")
            
        x += 1
        
        pbar.update(len(smooths)) #this adds a little symbol at each iteration
    pbar.finish()
    
    
    if save:
        pickle.dump(smooths, open("saved/test_smooths.p", "wb"))
        pickle.dump(S, open("saved/test_S.p", "wb"))
        #pickle.dump(, open("saved/test_S.p", "wb"))
    
    ## Gaussian elimination ....
    print("mod 2ing matrix")
    
    M = np.remainder(smooths, 2) #reduces the factors to a matrix mod 2 to find dependent rows
     
    print("reducing matrix")
   
    red_M, elim_rows = matrix_structure_hueristic(M)
    
    print("finding dependence")
    
    reduced_M, dep_rows = reduce_matrix(M)
    
    row_sum = [sum(elim_rows[:i]) for i in range(len(elim_rows))]
    
    rows = [row_sum.index(d) for d in dep_rows]
    print(rows)
    #rows contains the indices of M that sum to zero
    

    
    print("solving for x,y...")
    
    x = 1
    for r in rows:
        x = (x * S[r][0]) % n
    
    y = 1 #how to use a_i's??
    ys = []
    for j in range(K):
        p = sum([smooths[r][j] for r in rows])/2
        #y = (y * b_primes[j] ** p) % n
        #y = y * bin_ladder(b_primes[j], int(p), n)
        ys.append(bin_ladder(b_primes[j], int(p), n))
        #ys.append((b_primes[j] ** p) % n)
    
    y = 1
    for i in ys:
        y = (y * i) % n
    
    if verbose: print("y=", y)
    
    #y = y % n
    #y = 1
    #for a in a_s:
    #    #y *= bin_ladder(a, 1/2, n)
    #    y = (y * a**1/2) % n
    

    
    d = gcd(x-y, n)
    
    print(n, "=", d, "*", n/d)
    assert n % d == 0
    
    return d
        
        
    
    
quadratic_sieve(16921456439215439701, verbose=False, save=True)
#539873, B=19
#16921456439215439701
#46839566299936919234246726809

finding primes...


 39%|███▊      | 13720/35596 [00:00<00:00, 69335.97it/s]

finding a_i's....


100%|██████████| 35596/35596 [00:00<00:00, 63810.86it/s]
b-smooth count:   0% [                                        ] ETA:   0:01:33 

sieving...
4113569793


b-smooth count:  43% [-----------------                       ] ETA:   0:01:34 

In [ ]:
M = np.array([np.array([1, 0, 0, 0, 1, 0, 0, 0,]),
 np.array([0, 0, 0, 0, 1, 0, 1, 0]),
 [1, 0, 0, 0, 1, 1, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 1, 1, 0, 0,],
 [0, 1, 1, 0, 0, 0, 1, 0,],
 [1, 1, 1, 0, 0, 0, 0, 0,],
 [1, 0, 1, 1, 0, 1, 0, 0,],
 [1, 1, 0, 1, 1, 1, 0, 0]])

print(M)

In [ ]:
import timeit 
add_cols(M, 0, 1)

In [ ]:
M[:, 0] ^ M[:, 1]

In [ ]:
matrix_structure_hueristic(M, verbose=True)

In [ ]:
%timeit np.remainder(M, 2)

In [ ]:
%timeit [mod2(x) for x in M]

In [ ]:
a = np.array([1, 0, 1, 1, 1, 0, 1, 1, 1] )# ^ 
b = np.array([1, 1, 1, 1, 1, 0, 0, 0, 1])
print(type(a))

In [ ]:
%timeit a ^ b

In [ ]:
%timeit (a + b) % 2